## Global Historical Climatology Network Dataset
Variables are stored in both rows and columns
This dataset represents the daily weather records for a weather station (MX17004) in Mexico for five months in 2010.

In [100]:
import pandas as pd
import numpy as np
import datetime
import re

In [32]:
from google.cloud import bigquery
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./ironhack_service_account_big_query.json"
import six
six.moves.reload_module(six)
client = bigquery.Client()

In [121]:
df=pd.read_csv('../weather-raw.csv', header=0)
df.head()

,id,year,month,element,d1,d2,d3,d4,d5,d6,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.8,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.5,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,...,NaN,29.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,...,NaN,10.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
#col_df=list(filter(df.columns .startswith('d')
regex = re.compile(r'^d+')

vars_c= list(filter(regex.search, col_df))
print(vars_c)

['d1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9', 'd10', 'd11', 'd12', 'd13', 'd14', 'd15', 'd16', 'd17', 'd18', 'd19', 'd20', 'd21', 'd22', 'd23', 'd24', 'd25', 'd26', 'd27', 'd28', 'd29', 'd30', 'd31']


In [35]:
melted = pd.melt(df, id_vars = ['id','year','month','element'],
                value_vars = vars_c)
melted.head(10)

,id,year,month,element,variable,value
0,MX17004,2010,1,tmax,d1,NaN
1,MX17004,2010,1,tmin,d1,NaN
2,MX17004,2010,2,tmax,d1,NaN
3,MX17004,2010,2,tmin,d1,NaN
4,MX17004,2010,3,tmax,d1,NaN
5,MX17004,2010,3,tmin,d1,NaN
6,MX17004,2010,4,tmax,d1,NaN
7,MX17004,2010,4,tmin,d1,NaN
8,MX17004,2010,5,tmax,d1,NaN
9,MX17004,2010,5,tmin,d1,NaN


In [130]:
df_pivot=melted.pivot_table(index=['id','year','month','variable'], columns='element')
df_pivot.reset_index(inplace=True)
df_pivot.columns = [' '.join(col).strip() for col in df_pivot.columns.values]
df_pivot[['x','variable']] = df_pivot['variable'].str.split('d', n=1, expand=True)
df_pivot.rename(columns = {'variable':'day'}, inplace=True)
df_pivot.drop(columns='x', inplace=True)
df_pivot['year']=df_pivot['year'].astype('int64')
df_pivot['month']=df_pivot['month'].astype('int64')
df_pivot['day']= df_pivot['day'].astype('int64')

df_pivot['date'] = df_pivot.year.astype(str).str.cat(df_pivot.month.astype(str), sep='/').str.cat(df_pivot.day.astype(str), sep='/')
df_pivot['date'] = pd.to_datetime(df_pivot['date'], format = '%Y-%m-%d')

# df_pivot.to_datetime(pd.Series(str.concat(year,month,day,sep=''['05/23/2005']), format="%m/%d/%Y")
df_pivot.head()

,id,year,month,day,value tmax,value tmin,date
0,MX17004,2010,1,30,27.8,14.5,2010-01-30
1,MX17004,2010,2,11,29.7,13.4,2010-02-11
2,MX17004,2010,2,2,27.3,14.4,2010-02-02
3,MX17004,2010,2,23,29.9,10.7,2010-02-23
4,MX17004,2010,2,3,24.1,14.4,2010-02-03


In [131]:
df_pivot[df_pivot['value tmax']<df_pivot['value tmin']]

,id,year,month,day,value tmax,value tmin,date


In [132]:
df_pivot.columns

Index(['id', 'year', 'month', 'day', 'value tmax', 'value tmin', 'date'], dtype='object')

In [137]:
column_order = ['id', 'date', 'year', 'month', 'day', 'value tmin', 'value tmax']
df_pivot= df_pivot[column_order]

In [138]:
df_pivot.head()

,id,date,year,month,day,value tmin,value tmax
0,MX17004,2010-01-30,2010,1,30,14.5,27.8
1,MX17004,2010-02-11,2010,2,11,13.4,29.7
2,MX17004,2010-02-02,2010,2,2,14.4,27.3
3,MX17004,2010-02-23,2010,2,23,10.7,29.9
4,MX17004,2010-02-03,2010,2,3,14.4,24.1
